# Imports

In [1]:
import os
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Classes & Helpers

## Data

In [26]:
class Data():
    def __init__(self,
                 x_min, x_max,
                 t_min, t_max,
                 test_dir,
                 eps=1e-5,
                 device='cpu',
                 dtype=torch.float32):

        self.x_min = x_min
        self.x_max = x_max
        self.t_min = t_min
        self.t_max = t_max
        self.test_dir = test_dir
        self.eps = eps
        self.device = device
        self.dtype = dtype


    def _generate_random_numbers(self, min_, max_, N):
        return min_ + (max_ - min_) * torch.rand(size=(N,), dtype=self.dtype)


    # *** Create in-domain points ***
    def sample_domain(self, N_domain, x_min, x_max, t_min, t_max):
        # Random Grid
        x_domain = self._generate_random_numbers(x_min, x_max, N_domain)
        t_domain = self._generate_random_numbers(t_min, t_max, N_domain)
        domain_data = torch.stack((x_domain, t_domain), dim=1)
        return torch.tensor(domain_data, dtype=self.dtype, device=self.device, requires_grad=True)


    # *** Boundary Conditions ***
    def sample_boundary(self, Nt_bound):
        # Random boundary points
        t_bound = self._generate_random_numbers(self.t_min, self.t_max, Nt_bound)
        x_left = - torch.ones(1, dtype=self.dtype)
        x_right = torch.ones(1, dtype=self.dtype)

        bound_data_left = torch.stack(torch.meshgrid(x_left, t_bound)).view(2, -1).permute(1, 0)
        bound_data_right = torch.stack(torch.meshgrid(x_right, t_bound)).view(2, -1).permute(1, 0)
        bound_data = torch.cat([bound_data_left, bound_data_right]).requires_grad_(True).to(self.device)

        u_bound = torch.zeros(len(bound_data), 1, dtype=self.dtype, device=self.device)

        return bound_data, u_bound


    # *** Initial Condition ***
    def sample_initial(self, Nx_init, x_min, x_max):
        # Random initial points
        x_init = self._generate_random_numbers(x_min, x_max, Nx_init)
        t_init = torch.zeros(1, dtype=self.dtype)
        init_data = torch.stack(torch.meshgrid(x_init, t_init)).view(2, -1).permute(1, 0).requires_grad_(True).to(self.device)

        u_init = - torch.sin(math.pi * x_init).to(self.device)

        return init_data, u_init

    # *** Test set ***
    def sample_test(self):
        test_data = pd.read_csv(self.test_dir).dropna().to_numpy()
        return torch.tensor(test_data, dtype=self.dtype, device=self.device, requires_grad=True)

## Networks

In [27]:
class MLPBase(nn.Module):
    def __init__(self, layers, activation=nn.Tanh(), weight_init=None, bias_init=None, device='cpu'):
        super().__init__()
        self.layers = layers
        self.activation = activation
        self.weight_init = weight_init
        self.bias_init = bias_init

        dense_layers = [
            self.dense_layer(in_features=self.layers[i], out_features=self.layers[i + 1])
            for i in range(len(layers) - 2)]
        dense_layers.append(nn.Linear(in_features=self.layers[-2], out_features=self.layers[-1]))

        self.mlp = nn.Sequential(*dense_layers).to(device)

    def dense_layer(self, in_features, out_features):
        dense_layer = nn.Sequential(
            nn.Linear(in_features=in_features, out_features=out_features),
        )

        if self.weight_init is not None:
            self.weight_init(dense_layer[0].weight)

        if self.bias_init is not None:
            self.bias_init(dense_layer[0].bias)

        dense_layer.add_module("activation", self.activation)
        return dense_layer


class MLPAgent(MLPBase):
    def __init__(self, layers, activation=nn.Tanh(), weight_init=None, bias_init=None, device='cpu'):
        super().__init__(layers, activation, weight_init, bias_init, device)

    def forward(self, x):
        g_out = self.mlp(x)
        x_out = torch.tanh(g_out[:, 0].clone()).view(-1, 1)
        t_out = torch.sigmoid(g_out[:, 1].clone()).view(-1, 1)
        return torch.cat((x_out, t_out), dim=1)


class MLPPinn(MLPBase):
    def __init__(self, layers, activation=nn.Tanh(), weight_init=None, bias_init=None, device='cpu'):
        super().__init__(layers, activation, weight_init, bias_init, device)

    def forward(self, x):
        return self.mlp(x)

In [28]:
class CNNDense(nn.Module):
    def __init__(self,
                 num_batches,
                 num_filters,
                 kernel_size,
                 mlp_hidden_layers,
                 mlp_hidden_activation,
                 device='cpu',
                 dtype=torch.float32):

        super().__init__()
        self.stride = 2
        self.padding = 0
        self.dilation = 1
        self.dtype = dtype
        self.device = device

        base_layers = [
            nn.Conv1d(in_channels=5,
                      out_channels=num_filters,
                      kernel_size=kernel_size,
                      stride=self.stride),
            nn.BatchNorm1d(num_filters),
            nn.ReLU(),
            nn.Flatten()
        ]

        conv1d_output_size = int(np.floor(1 + (num_batches + 2*self.padding - self.dilation*(kernel_size - 1) - 1) / self.stride))
        mlp_in_features = conv1d_output_size * num_filters

        mlp_layers = [mlp_in_features] + mlp_hidden_layers + [2]

        self.cnn_layer = nn.Sequential(*base_layers).to(device)
        self.mlp = MLPAgent(layers=mlp_layers,
                            activation=mlp_hidden_activation,
                            device=device).to(device)


    def forward(self, x):
        cnn_output = self.cnn_layer(x)
        return self.mlp(cnn_output)

## PINN

In [40]:
class PINNBase():
    def __init__(self,
                 layers,
                 activation,
                 device):

        self.v = 0.01 / math.pi

        # Define the model
        self.model = MLPPinn(layers=layers,
                             activation=activation,
                             weight_init=lambda m: nn.init.xavier_normal_(m.data, nn.init.calculate_gain('tanh')),
                             bias_init=lambda m: nn.init.zeros_(m.data),
                             device=device)

        # Set the optimizers
        adam = torch.optim.Adam(self.model.parameters())
        lbfgs = torch.optim.LBFGS(self.model.parameters(),
                                  lr=1,
                                  max_iter=2000,
                                #   max_iter=5000,
                                  max_eval=None,
                                  tolerance_grad=1e-07,
                                  tolerance_change=1e-09,
                                  history_size=100,
                                  line_search_fn='strong_wolfe')

        self.optimizers = {"adam": adam, "lbfgs": lbfgs}

        # Set the Loss function
        self.criterion = nn.MSELoss()

        # Set the MAE criterion for test data only
        self.l1_loss = nn.L1Loss()


    def forward(self, x):
        return self.model(x)


    def grad(self, output, input):
        return torch.autograd.grad(
                    output, input,
                    grad_outputs=torch.ones_like(output),
                    retain_graph=True,
                    create_graph=True
                )[0]


    def calculate_pde_residual(self, x):
        # Forward pass
        u = self.forward(x)

        # Calculate 1st and 2nd derivatives
        du_dX = self.grad(u, x)
        du_dXX = self.grad(du_dX, x)

        # Retrieve the partial gradients
        du_dt = du_dX[:, 1].flatten()
        du_dx = du_dX[:, 0].flatten()
        du_dxx = du_dXX[:, 0].flatten()

        pde_res = du_dt + u.flatten() * du_dx - self.v * du_dxx

        return u, pde_res


    def calculate_pde_loss(self, data):
        # Calculate the domain loss
        _, self.pde_res = self.calculate_pde_residual(data)
        pde_target = torch.zeros_like(self.pde_res)
        return self.criterion(self.pde_res, pde_target)


    def calculate_total_loss(self, data):
        # Calculate boundary loss
        loss_b = self.criterion(
            self.forward(data["bound_data"]).flatten(),
            data["u_bound"].flatten()
        )
        # print(f"loss_b = {loss_b}")

        # Calculate initial loss
        loss_i = self.criterion(
            self.forward(data["init_data"]).flatten(),
            data["u_init"].flatten()
        )
        # print(f"loss_i = {loss_i}")

        # Calculate the domain loss
        domain_data = torch.cat((data["domain_data"], data["anchors"]), dim=0)
        loss_pde = self.calculate_pde_loss(domain_data)
        # print(f"loss_pde = {loss_pde}")

        # Calculate total discriminator loss
        return loss_b + loss_i + loss_pde


    def evaluate_pinn(self, test_data):
        _, pde_res = self.calculate_pde_residual(test_data)
        pde_target = torch.zeros_like(pde_res)
        return self.l1_loss(pde_res, pde_target)


    def train_step(self, data):
        loss = self.calculate_total_loss(data)
        loss.backward()
        return loss


    def closure(self):
        self.lbfgs_optimizer.zero_grad()
        return self.train_step(self.data)

In [41]:
class PINN():
    def __init__(self,
                 x_min, x_max,
                 t_min, t_max,
                 N_domain,
                 Nx_init,
                 Nt_bound,
                 test_dir,
                 general_max_episode_steps,
                 layers, activation,
                 checkpoint_path,
                 eps=1e-5,
                 device='cpu',
                 dtype=torch.float32):

        # Constants
        self.checkpoint_path = checkpoint_path
        self.device = device
        self.dtype = dtype
        self.x_min = x_min
        self.x_max = x_max
        self.t_min = t_min
        self.t_max = t_max
        self.general_max_episode_steps = general_max_episode_steps
        self.data = {}

        # Create real data
        self.data_init = Data(x_min, x_max,
                              t_min, t_max,
                              test_dir,
                              eps,
                              device,
                              dtype)

        # Create train data
        self.data["domain_data"] = self.data_init.sample_domain(N_domain, self.x_min, self.x_max, self.t_min, self.t_max)
        self.data["bound_data"], self.data["u_bound"] = self.data_init.sample_boundary(Nt_bound)
        self.data["init_data"], self.data["u_init"] = self.data_init.sample_initial(Nx_init, self.x_min, self.x_max)

        # Create test data
        self.test_data = self.data_init.sample_test()

        # Create base PINN
        self.base_pinn = PINNBase(layers, activation, device)


    def add_anchors(self, step, point):
        if step == 0:
            self.data["anchors"] = torch.empty(self.general_max_episode_steps, 2, device=self.device)
        else:
            new_anchors = torch.tensor(point, dtype=self.dtype, device=self.device, requires_grad=True).view(-1, 2)
            self.data["anchors"] = torch.cat((self.data["anchors"], new_anchors), dim=0)


    def train_with_adam(self, N_adam, data):
        optimizer = self.base_pinn.optimizers['adam']

        for epoch in range(1, N_adam + 1):
            optimizer.zero_grad()
            loss = self.base_pinn.train_step(data)
            optimizer.step()

        return loss


    def train_with_lbfgs(self, N_lbfgs, data):
        self.base_pinn.lbfgs_optimizer = self.base_pinn.optimizers["lbfgs"]
        self.base_pinn.data = data

        for epoch in range(1, N_lbfgs + 1):
            loss = self.base_pinn.lbfgs_optimizer.step(self.base_pinn.closure)

        return loss


    def checkpoint(self):
        torch.save({
            "model": self.base_pinn.model.state_dict()
        }, self.checkpoint_path)


    def format_loss(self, loss):
        if loss == 0:
            return "0.0e+00"

        # Calculate the exponent part
        exponent = int(math.log10(abs(loss)))

        # Determine the format based on the value of the loss
        if abs(loss) < 1:
            formatted_loss = f"{loss:.2e}"
        else:
            # Adjust the sign of the formatted loss
            sign = "-" if loss < 0 else ""

            # Calculate the number of decimal places
            decimal_places = 2 - exponent

            # Ensure at least two decimal places
            decimal_places = max(decimal_places, 2)

            # Format the loss with the correct sign
            formatted_loss = f"{sign}{abs(loss):.{decimal_places}e}"

        return formatted_loss


    def keep_checkpoints_and_print_losses(self, iter, patience, print_every, loss, loss_test):

        loss_str = self.format_loss(loss)
        loss_test_str = self.format_loss(loss_test)

        if iter == 1:
            self.best_val_loss = loss_test
            self.best_epoch = -1
            self.checkpoint()
            self.flag = 1
            print(f"Iteration: {iter} | loss: {loss_str} | test_mae: {loss_test_str} - *Checkpoint*")
        else:
            if loss_test < self.best_val_loss:
                self.best_val_loss = loss_test
                self.best_epoch = iter
                self.checkpoint()
                self.flag = 1
                if iter % print_every == 0:
                    print(f"Iteration: {iter} | loss: {loss_str} | test_mae: {loss_test_str} - *Checkpoint*")
            elif iter - self.best_epoch > patience:
                if iter % print_every == 0:
                    self.early_stopping_applied = 1
                    print(f"Iteration: {iter} | loss: {loss_str} | test_mae: {loss_test_str}")
                return

        if (self.flag == 0) and (iter % print_every == 0):
            print(f"Iteration: {iter} | loss: {loss_str} | test_mae: {loss_test_str}")


    def train(self, iters, patience, print_every, N_adam, N_lbfgs):
        print(f"\nPINN: {iters} iterations")
        print(f"a. PINN: {N_adam} epochs --> Adam")
        print(f"b. PINN: {N_lbfgs} epochs --> L-BFGS")

        for iter in tqdm(range(1, iters + 1)):
            self.flag = 0
            self.early_stopping_applied = 0

            # Train with adam
            print(f"\n\nTraining with ADAM...")
            loss = self.train_with_adam(N_adam, self.data)

            # Train with L-BFGS
            print(f"\nTraining with L-BFGS...")
            loss = self.train_with_lbfgs(N_lbfgs, self.data)

            # Evaluate on test
            loss_test = self.base_pinn.evaluate_pinn(self.test_data)

            # Keep check points and print losses
            self.keep_checkpoints_and_print_losses(iter, patience, print_every, loss, loss_test)
            if self.early_stopping_applied:
                print(f"\nEarly stopping applied at epoch {iter}.")
                break

        return loss_test.detach().cpu().numpy()

## Agent

In [31]:
class DeepQNetwork(nn.Module):
  def __init__(self, layers, activation, checkpoint_path):
    super(DeepQNetwork, self).__init__()

    self.checkpoint_path = checkpoint_path
    self.model = MLPAgent(layers=layers,
                          activation=activation,
                          weight_init=lambda m: nn.init.xavier_normal_(m.data),
                          bias_init=lambda m: nn.init.zeros_(m.data),
                          device=device)
    self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)
    self.criterion = nn.MSELoss()

  def forward(self, state):
    return self.model(state)

  def save_checkpoint(self):
    torch.save(self.model.state_dict(), self.checkpoint_path)

  def load_checkpoint(self):
    self.model.load_state_dict(torch.load(self.checkpoint_path))

In [52]:
class DDQNAgent():
    def __init__(self, pinn, iterations, patience, print_every,
                 num_epochs_adam, num_epochs_lbfgs, layers,
                 activation, max_anchors, gamma, epsilon,
                 eps_min, eps_dec, replace, checkpoint_path_eval,
                 checkpoint_path_next, device='cpu', dtype=torch.float32):

        self.pinn = pinn
        self.iterations = iterations
        self.patience = patience
        self.print_every = print_every
        self.num_epochs_adam = num_epochs_adam
        self.num_epochs_lbfgs = num_epochs_lbfgs

        self.gamma = gamma
        self.epsilon = epsilon
        self.eps_min = eps_min
        self.eps_dec = eps_dec
        self.replace_target_counter = replace
        self.max_anchors = max_anchors
        self.sampled_points = []
        self.Q_eval = DeepQNetwork(layers, activation, checkpoint_path_eval)
        self.Q_next = DeepQNetwork(layers, activation, checkpoint_path_next)

        self.step_counter = 0
        self.learn_step_counter = 0
        self.device = device
        self.dtype = dtype

    def create_random_action(self):
        x = 2 * torch.rand(1, 1, dtype=self.dtype, device=self.device) - 1
        t = torch.rand(1, 1, dtype=self.dtype, device=self.device)
        return torch.cat((x, t), dim=1)

    def choose_action(self, observation):
        return self.Q_eval.forward(observation) if np.random.random() > self.epsilon else self.create_random_action()

    def create_observation(self, action=None):
        if action is None:
            action = self.create_random_action()

        point = action.requires_grad_(True)
        u_hat, pde_res = self.pinn.base_pinn.calculate_pde_residual(point)
        norm_step_val = self.step_counter / self.max_anchors
        norm_step = norm_step_val * torch.ones(u_hat.shape[0], 1, dtype=self.dtype, device=self.device)
        return torch.cat((action, u_hat, pde_res.view(-1, 1), norm_step), dim=1).detach()

    def reset(self):
        self.step_counter = 0
        self.sampled_points = []
        self.pinn.add_anchors(step=self.step_counter, point=None)
        return self.create_observation(action=None)

    def step(self, action):
        self.step_counter += 1

        # Add sampled points to training data
        self.pinn.add_anchors(self.step_counter, action)
        self.sampled_points.append(action)

        # Generating next input
        next_obs = self.create_observation(action=action)

        # If batch is completed Then train, reward the agent and restart episode (sampling process).
        # Finally store log Else continue episode (sampling process) and no reward is provided
        if self.step_counter < self.max_anchors:
            reward = 0.0
            done = False
        else:
            self.eval_error = self.pinn.train(self.iterations,
                                              self.patience,
                                              self.print_every,
                                              self.num_epochs_adam,
                                              self.num_epochs_lbfgs)
            reward = float(self.eval_error)
            done = True

        # Return transition tuple: next observation, reward, done, truncate=False, info=None
        return next_obs, reward, done, {}


    def replace_target_network(self):
        if self.step_counter % self.replace_target_counter == 0:
            self.Q_next.load_state_dict(self.Q_eval.state_dict())


    def decrement_epsilon(self):
        self.epsilon = self.epsilon - self.eps_dec if self.epsilon > self.eps_min else self.eps_min


    def save_models(self):
        self.Q_eval.save_checkpoint()
        self.Q_next.save_checkpoint()


    def load_models(self):
        self.Q_eval.load_checkpoint()
        self.Q_next.load_checkpoint()


    def train(self, obs, reward, obs_, done):
        self.Q_eval.optimizer.zero_grad()

        self.replace_target_network()

        # indices = np.arange(self.batch_size)

        # ================== CHANGES START HERE ... =====================
        Q_pred = self.Q_eval.forward(obs)   #[indices, actions]
        Q_next = self.Q_next.forward(obs_)
        Q_eval = self.Q_eval.forward(obs_)
        # max_actions = T.argmax(Q_eval, dim=1)

        # where the episodes are reach a terminal state (done=1),
        # there target must be equal to r
        # Q_next[done] = 0.0
        if done:
            Q_target = reward + self.gamma * torch.zeros_like(Q_next)
        else:
            Q_target = reward + self.gamma * Q_next    #[indices, max_actions]
        # ================== CHANGES END HERE ... =======================

        loss = self.Q_eval.criterion(Q_target, Q_pred)  #.to(self.Q_eval.device)
        loss.backward()
        self.Q_eval.optimizer.step()
        self.step_counter += 1

        self.decrement_epsilon()
        self.learn_step_counter += 1

        return loss.item()

# MAIN

## Configuration

In [54]:
# General
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
dtype = torch.float32
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Data
x_min, x_max = -1, 1
t_min, t_max = 0, 1
N_domain = 2_500        # Number of domain training points
Nt_bound = 100           # Number of training points for x=-1 and x=1
Nx_init = 100            # Number of training points for t=0
test_dir = '/content/drive/MyDrive/test_data.csv' #'test_data.csv'

# PINN-Model
N_layers = 3
N_neurons = 20
layers = [2] + N_layers * [N_neurons] + [1]
hidden_activation = nn.Tanh()

# PINN-Training
iterations = 1
patience = iterations
print_every = 1
num_epochs_adam = 100
num_epochs_lbfgs = 1
chechpoint_path = "pinn_model.pth"

# Agent
trials = 100
max_anchors = 25
epsilon = 0.1
gamma = 0.99
eps_min = 0.01
eps_dec = 5e-7
replace = 50
checkpoint_path_eval="ddq_model_eval.pth",
checkpoint_path_next="ddq_model_next.pth"
N_agent_layers = 3
N_agent_neurons = 64
agent_layers = [5] + N_agent_layers * [N_agent_neurons] + [2]
agent_hidden_activation = nn.ReLU()

## Train

In [55]:
pinn = PINN(
    x_min, x_max, t_min, t_max,
    N_domain, Nx_init, Nt_bound,
    test_dir, max_anchors,
    layers, hidden_activation,
    chechpoint_path,
    device=device
)

agent = DDQNAgent(
    pinn, iterations, patience, print_every,
    num_epochs_adam, num_epochs_lbfgs, agent_layers,
    agent_hidden_activation, max_anchors, gamma, epsilon,
    eps_min, eps_dec, replace, checkpoint_path_eval,
    checkpoint_path_next, device, dtype
)

for trial in range(trials):
    print(f"\n******************** trial = {trial} ********************")
    obs = agent.reset().squeeze().view(-1, 5)
    score = 0
    done = False

    while not done:
        action = agent.choose_action(obs)
        obs_, reward, done, info = agent.step(action)
        score += reward
        agent_loss = agent.train(obs, reward, obs_, int(done))
        obs = obs_

    print(f"Agent loss: {agent_loss}")
    print(f"Score: {score} | epsilon: {agent.epsilon}")


******************** trial = 0 ********************

PINN: 1 iterations
a. PINN: 100 epochs --> Adam
b. PINN: 1 epochs --> L-BFGS


  0%|          | 0/1 [00:00<?, ?it/s]



Training with ADAM...

Training with L-BFGS...


100%|██████████| 1/1 [00:40<00:00, 40.76s/it]


Iteration: 1 | loss: 2.10e-01 | test_mae: 3.13e-02 - *Checkpoint*
Agent loss: 0.11416581273078918
Score: 0.03125103935599327 | epsilon: 0.0999935

******************** trial = 1 ********************

PINN: 1 iterations
a. PINN: 100 epochs --> Adam
b. PINN: 1 epochs --> L-BFGS


...

Iteration: 1 | loss: 9.31e-06 | test_mae: 3.52e-02 - *Checkpoint*
Agent loss: 0.051944855600595474
Score: 0.03524667024612427 | epsilon: 0.09935649999999936

******************** trial = 99 ********************

PINN: 1 iterations
a. PINN: 100 epochs --> Adam
b. PINN: 1 epochs --> L-BFGS


  0%|          | 0/1 [00:00<?, ?it/s]



Training with ADAM...

Training with L-BFGS...


100%|██████████| 1/1 [00:05<00:00,  5.07s/it]

Iteration: 1 | loss: 3.17e-05 | test_mae: 3.45e-02 - *Checkpoint*
Agent loss: 0.007982981391251087
Score: 0.03450756147503853 | epsilon: 0.09934999999999936
